#  1. Import Libraries


In [ ]:
# Import necessary libraries for Neural Networks (PyTorch), Randomness, and Visualization
import numpy as np           # For mathematical operations and arrays
import torch                 # PyTorch for building neural networks
import torch.nn as nn        # Neural network layers
import torch.optim as optim   # Optimizers for updating the weights
import random                # Randomness for city generation and action selection
import matplotlib.pyplot as plt  # For plotting and visualization


# 2. Traveling Salesman Problem (TSP)

The Traveling Salesman Problem (TSP) asks the question: "Given a list of cities and the distances between them, what is the shortest possible route that visits each city exactly once and returns to the origin city?"

We will solve this using Deep Reinforcement Learning with a Q-learning-based agent.


# 3. City Class for TSP


In [ ]:
# City Class
# This class represents each city in the TSP problem by its coordinates (x, y).
# It also includes a method to calculate the Euclidean distance to another city.

class City:
    def __init__(self, x, y):
        """
        Initialize the city with its x and y coordinates.
        """
        self.x = x
        self.y = y

    def distance_to(self, city):
        """
        Compute the Euclidean distance between this city and another city.
        Euclidean distance formula: sqrt((x2 - x1)^2 + (y2 - y1)^2)
        """
        return np.sqrt((self.x - city.x)**2 + (self.y - city.y)**2)


# 4. Deep Q-Network (DQN) Definition


In [ ]:
# Neural Network Model for Q-learning
# This model will approximate the Q-values for each possible action (city to visit next).
# The network has three fully connected layers with ReLU activations.

class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        """
        Initialize the Deep Q-Network with input_size representing the number of cities
        and output_size representing the number of possible actions (next city to visit).
        """
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)   # First hidden layer
        self.fc2 = nn.Linear(128, 64)           # Second hidden layer
        self.fc3 = nn.Linear(64, output_size)   # Output layer

    def forward(self, x):
        """
        Define the forward pass through the neural network.
        Input is the current state (cities visited so far), and output is Q-values.
        """
        x = torch.relu(self.fc1(x))   # Apply ReLU activation to first hidden layer
        x = torch.relu(self.fc2(x))   # Apply ReLU activation to second hidden layer
        return self.fc3(x)   # Output layer provides Q-values for each action (city)


# 5. TSP Agent Setup (Reinforcement Learning)


In [ ]:
# TSP Agent for Reinforcement Learning
# This agent interacts with the environment, makes decisions, stores experiences, and learns from them.
# The agent uses Q-learning to find the optimal tour that minimizes the travel distance.

class TSPAgent:
    def __init__(self, num_cities, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, lr=0.001):
        """
        Initialize the agent with hyperparameters for Q-learning:
        - gamma: discount factor (how much future rewards are considered)
        - epsilon: initial exploration rate (for epsilon-greedy strategy)
        - epsilon_decay: how quickly exploration rate decays
        - epsilon_min: minimum exploration rate
        - lr: learning rate for optimizer
        """
        self.num_cities = num_cities
        self.gamma = gamma  # Discount factor for future rewards
        self.epsilon = epsilon  # Exploration rate for epsilon-greedy policy
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.memory = []  # Memory buffer to store experiences
        self.batch_size = 32  # Number of experiences to sample for training
        self.memory_capacity = 10000  # Max capacity of memory buffer
        self.q_network = DQN(num_cities, num_cities)  # Initialize the Q-network
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)  # Optimizer for training
        self.loss_fn = nn.MSELoss()  # Mean Squared Error loss for training


# 6. Agent: Experience Storage and Action Selection


In [ ]:
    def store_experience(self, state, action, reward, next_state, done):
        """
        Store the agent's experience (state, action, reward, next_state, done) in memory for experience replay.
        If memory exceeds its capacity, the oldest experience is removed.
        """
        if len(self.memory) > self.memory_capacity:
            self.memory.pop(0)  # Remove the oldest experience if memory is full
        self.memory.append((state, action, reward, next_state, done))  # Add new experience

    def choose_action(self, state):
        """
        Choose an action (next city to visit) using epsilon-greedy strategy.
        - With probability epsilon, choose a random action (exploration).
        - Otherwise, choose the action with the highest Q-value (exploitation).
        """
        if np.random.rand() < self.epsilon:  # Exploration
            return random.randint(0, self.num_cities - 1)  # Choose a random city to visit
        else:  # Exploitation
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert state to tensor
            with torch.no_grad():  # No gradient calculation needed
                q_values = self.q_network(state_tensor)  # Get Q-values for current state
            return torch.argmax(q_values).item()  # Choose the action with the highest Q-value


# 7. Training the Agent (Q-Learning)


In [ ]:
    def train(self):
        """
        Train the Q-network using the experiences stored in memory.
        For each experience, update the Q-value towards the target value:
        Q_target = reward + gamma * max(Q(next_state)) if not done.
        """
        if len(self.memory) < self.batch_size:
            return  # Don't train if there aren't enough experiences in memory

        # Sample a batch of experiences randomly from memory
        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in batch:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert state to tensor
            next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)  # Convert next state to tensor
            target = reward
            if not done:
                target = reward + self.gamma * torch.max(self.q_network(next_state_tensor))  # Target Q-value

            q_value = self.q_network(state_tensor)[0, action]  # Current Q-value for the action
            loss = self.loss_fn(q_value, torch.tensor(target))  # Compute the loss (target - predicted)

            # Perform backpropagation to update network weights
            self.optimizer.zero_grad()  # Reset gradients
            loss.backward()  # Compute gradients
            self.optimizer.step()  # Update network weights

        # Decay epsilon for less exploration over time
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# 8. TSP Environment Class (Simulating the Problem)


In [ ]:
# TSP Environment
# This environment simulates the cities and their connections.
# It interacts with the agent and provides rewards based on actions (city visits).

class TSPEnvironment:
    def __init__(self, cities):
        """
        Initialize the environment with a list of cities.
        It also tracks the current state (cities visited), total distance, and the current city.
        """
        self.cities = cities
        self.num_cities = len(cities)
        self.current_state = []  # The current state will store which cities have been visited
        self.current_city = None  # The current city the agent is located in
        self.visited = []  # List of visited cities
        self.total_distance = 0  # Total distance traveled by the agent

    def reset(self):
        """
        Reset the environment for a new episode.
        The agent starts from a random city, and no cities have been visited yet.
        """
        self.current_state = np.zeros(self.num_cities)  # All cities are unvisited initially
        self.current_city = random.randint(0, self.num_cities - 1)  # Start from a random city
        self.visited = [self.current_city]  # Mark the start city as visited
        self.total_distance = 0  # Reset total distance traveled
        return self.current_state  # Return the initial state (unvisited cities)


# 9. Step Function for Environment (Reward Calculation)


In [ ]:
    def step(self, action):
        """
        Take a step in the environment by visiting a city (the action).
        - If the agent revisits a city, it gets a negative reward.
        - Otherwise, the reward is the negative distance traveled to the new city.
        - The episode ends when all cities are visited (done=True).
        """
        if action in self.visited:
            reward = -10  # Negative reward for revisiting a city
        else:
            reward = -self.cities[self.current_city].distance_to(self.cities[action])  # Negative distance as reward
            self.total_distance += abs(reward)  # Accumulate the total distance
            self.visited.append(action)  # Mark the city as visited

        self.current_city = action  # Move to the new city
        self.current_state[action] = 1  # Update the state to mark this city as visited

        done = len(self.visited) == self.num_cities  # Check if all cities have been visited
        if done:
            # Add the return trip to the starting city
            reward -= self.cities[self.current_city].distance_to(self.cities[self.visited[0]])
            self.total_distance += abs(reward)  # Include the distance to return to the start

        return self.current_state, reward, done  # Return the next state, reward, and done flag


# 10. Training Function for the TSP Agent


In [ ]:
# Function to train the agent for multiple episodes of interaction with the environment
def train_tsp_agent(cities, episodes=1000):
    """
    Train the TSP agent using Q-learning for a given number of episodes.
    The agent interacts with the environment, learns from its experiences, and improves its policy.
    """
    env = TSPEnvironment(cities)  # Create the environment with the cities
    agent = TSPAgent(num_cities=len(cities))  # Initialize the Q-learning agent

    for episode in range(episodes):
        state = env.reset()  # Reset the environment at the beginning of each episode
        done = False
        total_reward = 0  # Track total reward (negative total distance)

        while not done:
            action = agent.choose_action(state)  # Agent chooses the next city (action)
            next_state, reward, done = env.step(action)  # Environment responds with next state and reward
            agent.store_experience(state, action, reward, next_state, done)  # Store experience for replay
            state = next_state  # Move to the next state
            total_reward += reward  # Accumulate the reward (minimizing negative distance)
            agent.train()  # Train the agent based on stored experiences

        print(f"Episode {episode+1}, Total Reward: {total_reward}, Total Distance: {env.total_distance}")


# Plotting the solution learned by the agent
def plot_solution(cities, path):
    """
    Visualize the final path learned by the agent. The cities will be connected based on the path found.
    """
    x_coords = [city.x for city in path]  # Get x coordinates of the cities
    y_coords = [city.y for city in path]  # Get y coordinates of the cities
    plt.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'bo-')  # Plot path and return to start
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Traveling Salesman Path')
    plt.show()


# 12. Example Execution


In [ ]:
# Example execution: Define a set of random cities and train the TSP agent

# Generate a random set of cities for the TSP problem
num_cities = 10
cities = [City(x=random.randint(0, 100), y=random.randint(0, 100)) for _ in range(num_cities)]

# Train the Q-learning agent for 500 episodes
train_tsp_agent(cities, episodes=500)

# Plot the learned solution (after training, path is assumed to be obtained)
# plot_solution(cities, learned_path)  # This will visualize the path learned by the agent
